In [9]:
import os
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper
from langchain_tavily import TavilySearch
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")



USER_AGENT environment variable not set, consider setting it to identify your requests.


In [10]:
url= ["https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
      "https://lilianweng.github.io/posts/2023-06-23-agent/",
      "https://lilianweng.github.io/posts/2023-01-10-inference-optimization/"]


docs =[WebBaseLoader(url).load() for url in url]


In [ ]:
print(docs)

In [12]:
doc_list = [item for sublist in docs for item in sublist]

In [ ]:
print(doc_list)

In [18]:
embedding = OpenAIEmbeddings()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)

doc_split = text_splitter.split_documents(doc_list)

vectirestore=FAISS.from_documents(documents= doc_split,embedding=embedding)

In [20]:
retriver = vectirestore.as_retriever()

In [ ]:
response = retriver.invoke("What is a prompt")
response

In [27]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

class RoutQuery(BaseModel):
    """Route a user query to the most relevant datasource"""

    document_source : Literal["vectorestore","Websearch"] = Field(
        ...,
        description="Given a user question choose to route it to a web search or a vectorstore search"
    )

llm= ChatOpenAI(model="gpt-4o-mini",temperature=0)
structure_output =llm.with_structured_output(RoutQuery)


#Prompt

system = """You are an expert at routing a user question to a vectorestore or websearch. The Vectorestore contains documents related to agents, 
prompt engineering and adversarial attacks.User the vectorestore for question on these topics. Otherwise, use Websearch"""

route_prompt = ChatPromptTemplate.from_messages(
 
 [
     ("system",system),
     ("human", "{question}")
 ]
)

question_router = route_prompt|structure_output

print(question_router.invoke({"question":" who won the cricket world cup 2023"}))

document_source='Websearch'
